# PRSM Economic Model Dashboard

**Interactive Jupyter Dashboard for Phase 2 Economic Validation**

This dashboard provides comprehensive scenario analysis and economic modeling capabilities for the PRSM tokenomics system. It enables exploration of:

- **Agent-Based Economic Simulation** with 10K+ agents across 4 stakeholder types
- **Market Condition Analysis** under different economic scenarios
- **Price Discovery Validation** and stability testing
- **Network Effects Modeling** and stakeholder behavior analysis
- **Economic Sustainability Metrics** and validation targets

## Dashboard Features:
1. Real-time economic simulation visualization
2. Interactive parameter adjustment
3. Comparative scenario analysis
4. Phase 2 validation reporting
5. Economic recommendations and insights

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import asyncio
import warnings
warnings.filterwarnings('ignore')

# Import PRSM economic model
import sys
sys.path.append('../')
from prsm.economics.agent_based_model import (
    PRSMEconomicModel, 
    EconomicSimulationRunner,
    MarketCondition,
    StakeholderType,
    run_economic_simulation,
    run_comprehensive_economic_validation
)

print("📊 PRSM Economic Dashboard Loaded Successfully!")
print("Ready for interactive economic analysis and scenario modeling.")

## 1. Economic Simulation Controls

Interactive controls for running economic simulations with different parameters:

In [ ]:
# Simulation control widgets
simulation_widgets = {
    'num_agents': widgets.IntSlider(
        value=1000,
        min=100,
        max=10000,
        step=100,
        description='Agents:',
        style={'description_width': 'initial'}
    ),
    'simulation_steps': widgets.IntSlider(
        value=168,
        min=24,
        max=720,
        step=24,
        description='Steps (Hours):',
        style={'description_width': 'initial'}
    ),
    'market_condition': widgets.Dropdown(
        options=[('Stable', 'stable'), ('Bull Market', 'bull'), ('Bear Market', 'bear'), ('Volatile', 'volatile')],
        value='stable',
        description='Market Condition:',
        style={'description_width': 'initial'}
    ),
    'initial_token_supply': widgets.FloatText(
        value=1000000.0,
        description='Token Supply:',
        style={'description_width': 'initial'}
    )
}

# Display controls
display(HTML("<h3>🎛️ Simulation Parameters</h3>"))
control_box = widgets.VBox([
    widgets.HBox([simulation_widgets['num_agents'], simulation_widgets['simulation_steps']]),
    widgets.HBox([simulation_widgets['market_condition'], simulation_widgets['initial_token_supply']])
])
display(control_box)

# Global variables for simulation results
current_simulation_results = None
current_model = None

## 2. Run Economic Simulation

Execute the agent-based economic simulation with selected parameters:

In [ ]:
async def run_simulation_with_params():
    """Run economic simulation with current widget parameters"""
    global current_simulation_results, current_model
    
    # Get parameters from widgets
    num_agents = simulation_widgets['num_agents'].value
    steps = simulation_widgets['simulation_steps'].value
    market_condition_str = simulation_widgets['market_condition'].value
    initial_supply = simulation_widgets['initial_token_supply'].value
    
    # Convert market condition string to enum
    market_condition_map = {
        'stable': MarketCondition.STABLE,
        'bull': MarketCondition.BULL,
        'bear': MarketCondition.BEAR,
        'volatile': MarketCondition.VOLATILE
    }
    market_condition = market_condition_map[market_condition_str]
    
    print(f"🚀 Running simulation: {num_agents} agents, {steps} steps, {market_condition_str} market")
    
    # Create and run model
    from decimal import Decimal
    current_model = PRSMEconomicModel(
        num_agents=num_agents,
        initial_token_supply=Decimal(str(initial_supply)),
        market_condition=market_condition
    )
    
    # Run simulation
    for step in range(steps):
        current_model.step()
        if step % 24 == 0:  # Progress update every 24 hours
            print(f"   Day {step//24}: Price ${float(current_model.token_price):.2f}")
    
    # Get final results
    economic_summary = current_model.get_economic_summary()
    validation_results = current_model.validate_economic_targets()
    
    current_simulation_results = {
        'economic_summary': economic_summary,
        'validation_results': validation_results,
        'model': current_model,
        'parameters': {
            'num_agents': num_agents,
            'steps': steps,
            'market_condition': market_condition_str,
            'initial_supply': initial_supply
        }
    }
    
    print(f"✅ Simulation completed!")
    print(f"   Final Price: ${float(current_model.token_price):.2f}")
    print(f"   Network Value: ${float(current_model.network_value):,.0f}")
    print(f"   Validation Success: {validation_results.get('overall_success', False)}")
    
    return current_simulation_results

# Button to run simulation
run_button = widgets.Button(
    description='▶️ Run Economic Simulation',
    button_style='success',
    layout={'width': '250px', 'height': '40px'}
)

output_area = widgets.Output()

def on_run_button_clicked(b):
    with output_area:
        clear_output(wait=True)
        # Run simulation in async context
        import nest_asyncio
        nest_asyncio.apply()
        loop = asyncio.get_event_loop()
        results = loop.run_until_complete(run_simulation_with_params())

run_button.on_click(on_run_button_clicked)

display(run_button)
display(output_area)

## 3. Economic Metrics Visualization

Real-time visualization of key economic metrics:

In [ ]:
def create_economic_metrics_dashboard():
    """Create comprehensive economic metrics visualization"""
    if current_simulation_results is None:
        print("⚠️ Please run a simulation first!")
        return
    
    model = current_simulation_results['model']
    economic_summary = current_simulation_results['economic_summary']
    
    # Extract time series data from model
    metrics_data = model.economic_metrics
    
    if not metrics_data:
        print("⚠️ No metrics data available")
        return
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            '📈 Token Price Over Time',
            '💰 Network Value Growth', 
            '📊 Stakeholder Satisfaction',
            '⚖️ Wealth Distribution (Gini)'
        ),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # Extract time series data
    timestamps = [m.timestamp for m in metrics_data]
    token_prices = [float(m.average_price) for m in metrics_data]
    network_values = [float(m.network_value) for m in metrics_data]
    gini_coefficients = [m.gini_coefficient for m in metrics_data]
    
    # 1. Token Price Chart
    fig.add_trace(
        go.Scatter(
            x=list(range(len(token_prices))),
            y=token_prices,
            mode='lines+markers',
            name='Token Price',
            line=dict(color='#2E86AB', width=3),
            marker=dict(size=4)
        ),
        row=1, col=1
    )
    
    # 2. Network Value Chart
    fig.add_trace(
        go.Scatter(
            x=list(range(len(network_values))),
            y=network_values,
            mode='lines+markers',
            name='Network Value',
            line=dict(color='#A23B72', width=3),
            marker=dict(size=4)
        ),
        row=1, col=2
    )
    
    # 3. Stakeholder Satisfaction
    latest_satisfaction = metrics_data[-1].stakeholder_satisfaction
    stakeholder_types = list(latest_satisfaction.keys())
    satisfaction_values = [latest_satisfaction[st] * 100 for st in stakeholder_types]
    
    fig.add_trace(
        go.Bar(
            x=stakeholder_types,
            y=satisfaction_values,
            name='Satisfaction %',
            marker_color=['#F18F01', '#C73E1D', '#2E86AB', '#A23B72']
        ),
        row=2, col=1
    )
    
    # 4. Gini Coefficient (Wealth Distribution)
    fig.add_trace(
        go.Scatter(
            x=list(range(len(gini_coefficients))),
            y=gini_coefficients,
            mode='lines+markers',
            name='Gini Coefficient',
            line=dict(color='#F18F01', width=3),
            marker=dict(size=4)
        ),
        row=2, col=2
    )
    
    # Update layout
    fig.update_layout(
        title={
            'text': f"📊 PRSM Economic Dashboard - {current_simulation_results['parameters']['market_condition'].title()} Market",
            'x': 0.5,
            'font': {'size': 20}
        },
        height=800,
        showlegend=False,
        template='plotly_white'
    )
    
    # Update axis labels
    fig.update_xaxes(title_text="Time Steps (Hours)", row=1, col=1)
    fig.update_xaxes(title_text="Time Steps (Hours)", row=1, col=2)
    fig.update_xaxes(title_text="Stakeholder Type", row=2, col=1)
    fig.update_xaxes(title_text="Time Steps (Hours)", row=2, col=2)
    
    fig.update_yaxes(title_text="Price ($)", row=1, col=1)
    fig.update_yaxes(title_text="Value ($)", row=1, col=2)
    fig.update_yaxes(title_text="Satisfaction (%)", row=2, col=1)
    fig.update_yaxes(title_text="Gini Coefficient", row=2, col=2)
    
    return fig

# Button to create visualization
viz_button = widgets.Button(
    description='📊 Generate Economic Metrics Dashboard',
    button_style='info',
    layout={'width': '300px', 'height': '40px'}
)

viz_output = widgets.Output()

def on_viz_button_clicked(b):
    with viz_output:
        clear_output(wait=True)
        fig = create_economic_metrics_dashboard()
        if fig:
            fig.show()

viz_button.on_click(on_viz_button_clicked)

display(viz_button)
display(viz_output)

## 4. Scenario Comparison Analysis

Compare multiple market scenarios side-by-side:

In [ ]:
# Storage for scenario results
scenario_results = {}

async def run_scenario_comparison():
    """Run comprehensive scenario comparison across all market conditions"""
    global scenario_results
    
    print("🔄 Running comprehensive scenario analysis...")
    print("This will simulate all market conditions with current agent settings.")
    
    num_agents = simulation_widgets['num_agents'].value
    steps = min(simulation_widgets['simulation_steps'].value, 168)  # Cap at 1 week for comparison
    
    scenarios = {
        'Stable Market': MarketCondition.STABLE,
        'Bull Market': MarketCondition.BULL,
        'Bear Market': MarketCondition.BEAR,
        'Volatile Market': MarketCondition.VOLATILE
    }
    
    scenario_results = {}
    
    for scenario_name, market_condition in scenarios.items():
        print(f"   Running {scenario_name}...")
        
        # Create model for this scenario
        from decimal import Decimal
        model = PRSMEconomicModel(
            num_agents=num_agents,
            initial_token_supply=Decimal('1000000'),
            market_condition=market_condition
        )
        
        # Run simulation
        for step in range(steps):
            model.step()
        
        # Collect results
        economic_summary = model.get_economic_summary()
        validation_results = model.validate_economic_targets()
        
        scenario_results[scenario_name] = {
            'model': model,
            'economic_summary': economic_summary,
            'validation_results': validation_results,
            'final_price': float(model.token_price),
            'final_network_value': float(model.network_value),
            'price_stability': economic_summary['token_economics']['price_stability'],
            'avg_satisfaction': sum(economic_summary['network_health']['stakeholder_satisfaction'].values()) / 4
        }
        
        print(f"     ✅ {scenario_name}: Price ${scenario_results[scenario_name]['final_price']:.2f}")
    
    print("✅ Scenario comparison completed!")
    return scenario_results

def create_scenario_comparison_charts():
    """Create comparison charts for all scenarios"""
    if not scenario_results:
        print("⚠️ Please run scenario comparison first!")
        return
    
    # Create comparison charts
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            '💰 Final Token Prices by Scenario',
            '📊 Price Stability Comparison',
            '😊 Average Stakeholder Satisfaction',
            '✅ Phase 2 Validation Results'
        )
    )
    
    scenarios = list(scenario_results.keys())
    final_prices = [scenario_results[s]['final_price'] for s in scenarios]
    price_stabilities = [scenario_results[s]['price_stability'] for s in scenarios]
    avg_satisfactions = [scenario_results[s]['avg_satisfaction'] * 100 for s in scenarios]
    validation_successes = [scenario_results[s]['validation_results'].get('overall_success', False) for s in scenarios]
    
    colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D']
    
    # 1. Final Token Prices
    fig.add_trace(
        go.Bar(
            x=scenarios,
            y=final_prices,
            name='Final Price',
            marker_color=colors
        ),
        row=1, col=1
    )
    
    # 2. Price Stability
    fig.add_trace(
        go.Bar(
            x=scenarios,
            y=[s * 100 for s in price_stabilities],  # Convert to percentage
            name='Price Stability %',
            marker_color=colors
        ),
        row=1, col=2
    )
    
    # 3. Average Satisfaction
    fig.add_trace(
        go.Bar(
            x=scenarios,
            y=avg_satisfactions,
            name='Avg Satisfaction %',
            marker_color=colors
        ),
        row=2, col=1
    )
    
    # 4. Validation Success
    success_colors = ['green' if success else 'red' for success in validation_successes]
    fig.add_trace(
        go.Bar(
            x=scenarios,
            y=[100 if success else 0 for success in validation_successes],
            name='Validation Success',
            marker_color=success_colors
        ),
        row=2, col=2
    )
    
    # Update layout
    fig.update_layout(
        title={
            'text': "🔍 PRSM Economic Scenario Comparison Analysis",
            'x': 0.5,
            'font': {'size': 20}
        },
        height=800,
        showlegend=False,
        template='plotly_white'
    )
    
    # Update axis labels
    fig.update_yaxes(title_text="Price ($)", row=1, col=1)
    fig.update_yaxes(title_text="Stability (%)", row=1, col=2)
    fig.update_yaxes(title_text="Satisfaction (%)", row=2, col=1)
    fig.update_yaxes(title_text="Success (%)", row=2, col=2)
    
    return fig

# Scenario comparison controls
scenario_button = widgets.Button(
    description='🔄 Run Scenario Comparison',
    button_style='warning',
    layout={'width': '250px', 'height': '40px'}
)

scenario_viz_button = widgets.Button(
    description='📈 Show Comparison Charts',
    button_style='info',
    layout={'width': '250px', 'height': '40px'}
)

scenario_output = widgets.Output()

def on_scenario_button_clicked(b):
    with scenario_output:
        clear_output(wait=True)
        import nest_asyncio
        nest_asyncio.apply()
        loop = asyncio.get_event_loop()
        results = loop.run_until_complete(run_scenario_comparison())

def on_scenario_viz_button_clicked(b):
    with scenario_output:
        clear_output(wait=True)
        fig = create_scenario_comparison_charts()
        if fig:
            fig.show()

scenario_button.on_click(on_scenario_button_clicked)
scenario_viz_button.on_click(on_scenario_viz_button_clicked)

display(widgets.HBox([scenario_button, scenario_viz_button]))
display(scenario_output)

## 5. Phase 2 Validation Report

Comprehensive validation report for Phase 2 economic targets:

In [ ]:
def generate_phase2_validation_report():
    """Generate comprehensive Phase 2 validation report"""
    if not scenario_results:
        print("⚠️ Please run scenario comparison first to generate validation report!")
        return
    
    print("=" * 80)
    print("📋 PRSM PHASE 2 ECONOMIC VALIDATION REPORT")
    print("=" * 80)
    
    # Phase 2 Targets
    targets = {
        'price_stability': 0.8,  # 80% stability (±20% variance)
        'incentive_alignment': 0.8,  # 80% participant satisfaction
        'network_sustainability': 0.0,  # Positive growth rate
        'wealth_distribution': 0.7  # Gini coefficient < 0.7
    }
    
    print("\n🎯 VALIDATION TARGETS:")
    print(f"   • Price Stability: ≥{targets['price_stability']:.0%} (±20% variance max)")
    print(f"   • Incentive Alignment: ≥{targets['incentive_alignment']:.0%} participant satisfaction")
    print(f"   • Network Sustainability: Positive growth rate")
    print(f"   • Wealth Distribution: Gini coefficient ≤{targets['wealth_distribution']:.1f}")
    
    print("\n📊 SCENARIO RESULTS:")
    
    scenario_summary = []
    
    for scenario_name, results in scenario_results.items():
        validation = results['validation_results']
        summary = results['economic_summary']
        
        # Extract key metrics
        price_stability = results['price_stability']
        avg_satisfaction = results['avg_satisfaction']
        network_growth = summary['network_health'].get('average_growth_rate', 0)
        gini_coeff = summary['network_health']['gini_coefficient']
        overall_success = validation.get('overall_success', False)
        
        scenario_summary.append({
            'scenario': scenario_name,
            'price_stability': price_stability,
            'satisfaction': avg_satisfaction,
            'growth': network_growth,
            'gini': gini_coeff,
            'success': overall_success
        })
        
        # Individual scenario report
        print(f"\n   📈 {scenario_name}:")
        print(f"      Final Price: ${results['final_price']:.2f}")
        print(f"      Price Stability: {price_stability:.1%} {'✅' if price_stability >= targets['price_stability'] else '❌'}")
        print(f"      Avg Satisfaction: {avg_satisfaction:.1%} {'✅' if avg_satisfaction >= targets['incentive_alignment'] else '❌'}")
        print(f"      Network Growth: {network_growth:.1%} {'✅' if network_growth >= targets['network_sustainability'] else '❌'}")
        print(f"      Gini Coefficient: {gini_coeff:.3f} {'✅' if gini_coeff <= targets['wealth_distribution'] else '❌'}")
        print(f"      Overall Success: {'✅ PASSED' if overall_success else '❌ FAILED'}")
    
    # Aggregate analysis
    total_scenarios = len(scenario_summary)
    successful_scenarios = sum(1 for s in scenario_summary if s['success'])
    
    avg_price_stability = sum(s['price_stability'] for s in scenario_summary) / total_scenarios
    avg_satisfaction = sum(s['satisfaction'] for s in scenario_summary) / total_scenarios
    avg_growth = sum(s['growth'] for s in scenario_summary) / total_scenarios
    avg_gini = sum(s['gini'] for s in scenario_summary) / total_scenarios
    
    print("\n📈 AGGREGATE PERFORMANCE:")
    print(f"   Scenarios Passed: {successful_scenarios}/{total_scenarios} ({successful_scenarios/total_scenarios:.1%})")
    print(f"   Average Price Stability: {avg_price_stability:.1%}")
    print(f"   Average Satisfaction: {avg_satisfaction:.1%}")
    print(f"   Average Growth Rate: {avg_growth:.1%}")
    print(f"   Average Gini Coefficient: {avg_gini:.3f}")
    
    # Phase 2 pass criteria
    phase2_criteria = {
        'scenario_success_rate': successful_scenarios / total_scenarios >= 0.75,  # 75% scenarios must pass
        'avg_price_stability': avg_price_stability >= targets['price_stability'],
        'avg_satisfaction': avg_satisfaction >= targets['incentive_alignment'],
        'sustainable_growth': avg_growth >= targets['network_sustainability']
    }
    
    phase2_passed = sum(phase2_criteria.values()) >= 3  # At least 3/4 criteria must pass
    
    print("\n🏆 PHASE 2 VALIDATION RESULTS:")
    print(f"   Scenario Success Rate: {'✅' if phase2_criteria['scenario_success_rate'] else '❌'} ({successful_scenarios/total_scenarios:.1%} ≥ 75%)")
    print(f"   Price Stability Target: {'✅' if phase2_criteria['avg_price_stability'] else '❌'} ({avg_price_stability:.1%} ≥ {targets['price_stability']:.0%})")
    print(f"   Satisfaction Target: {'✅' if phase2_criteria['avg_satisfaction'] else '❌'} ({avg_satisfaction:.1%} ≥ {targets['incentive_alignment']:.0%})")
    print(f"   Sustainability Target: {'✅' if phase2_criteria['sustainable_growth'] else '❌'} ({avg_growth:.1%} ≥ 0%)")
    
    print("\n" + "=" * 80)
    if phase2_passed:
        print("🎉 PHASE 2 ECONOMIC VALIDATION: PASSED")
        print("The PRSM tokenomics system demonstrates economic viability!")
    else:
        print("⚠️ PHASE 2 ECONOMIC VALIDATION: NEEDS IMPROVEMENT")
        print("The tokenomics system requires optimization before Phase 2 completion.")
    print("=" * 80)
    
    # Recommendations
    print("\n💡 RECOMMENDATIONS:")
    if not phase2_criteria['avg_price_stability']:
        print("   • Implement price stabilization mechanisms (e.g., automated market makers)")
    if not phase2_criteria['avg_satisfaction']:
        print("   • Optimize reward distribution algorithms to improve stakeholder satisfaction")
    if not phase2_criteria['sustainable_growth']:
        print("   • Enhance value accrual mechanisms and network effects incentives")
    if not phase2_criteria['scenario_success_rate']:
        print("   • Improve system robustness across diverse market conditions")
    
    return {
        'phase2_passed': phase2_passed,
        'criteria': phase2_criteria,
        'scenario_summary': scenario_summary,
        'aggregate_metrics': {
            'avg_price_stability': avg_price_stability,
            'avg_satisfaction': avg_satisfaction,
            'avg_growth': avg_growth,
            'avg_gini': avg_gini
        }
    }

# Button to generate validation report
report_button = widgets.Button(
    description='📋 Generate Phase 2 Validation Report',
    button_style='success',
    layout={'width': '300px', 'height': '40px'}
)

report_output = widgets.Output()

def on_report_button_clicked(b):
    with report_output:
        clear_output(wait=True)
        validation_report = generate_phase2_validation_report()

report_button.on_click(on_report_button_clicked)

display(report_button)
display(report_output)

## 6. Economic Model Insights & Analytics

Deep dive into agent behaviors and economic patterns:

In [ ]:
def create_agent_behavior_analysis():
    """Analyze agent behaviors and economic patterns"""
    if current_simulation_results is None:
        print("⚠️ Please run a simulation first!")
        return
    
    model = current_simulation_results['model']
    
    # Collect agent data
    agent_data = {
        'content_creator': [],
        'query_user': [],
        'node_operator': [],
        'token_holder': []
    }
    
    for agent in model.schedule.agents:
        agent_type = agent.profile.agent_type.value
        agent_data[agent_type].append({
            'balance': float(agent.ftns_balance),
            'reputation': agent.reputation_score,
            'satisfaction': agent.satisfaction_score,
            'total_earnings': float(agent.total_earnings),
            'total_spending': float(agent.total_spending),
            'transactions': len(agent.transaction_history)
        })
    
    # Create analysis dashboard
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            '💰 Token Balance Distribution by Stakeholder',
            '⭐ Reputation Scores by Type',
            '😊 Satisfaction Levels',
            '📈 Earnings vs Spending Analysis'
        )
    )
    
    colors = {'content_creator': '#2E86AB', 'query_user': '#A23B72', 'node_operator': '#F18F01', 'token_holder': '#C73E1D'}
    
    # 1. Token Balance Distribution
    for agent_type, data in agent_data.items():
        if data:
            balances = [d['balance'] for d in data]
            fig.add_trace(
                go.Box(
                    y=balances,
                    name=agent_type.replace('_', ' ').title(),
                    marker_color=colors[agent_type]
                ),
                row=1, col=1
            )
    
    # 2. Reputation Scores
    for agent_type, data in agent_data.items():
        if data:
            reputations = [d['reputation'] for d in data]
            fig.add_trace(
                go.Box(
                    y=reputations,
                    name=agent_type.replace('_', ' ').title(),
                    marker_color=colors[agent_type],
                    showlegend=False
                ),
                row=1, col=2
            )
    
    # 3. Satisfaction Levels
    for agent_type, data in agent_data.items():
        if data:
            satisfactions = [d['satisfaction'] * 100 for d in data]
            fig.add_trace(
                go.Box(
                    y=satisfactions,
                    name=agent_type.replace('_', ' ').title(),
                    marker_color=colors[agent_type],
                    showlegend=False
                ),
                row=2, col=1
            )
    
    # 4. Earnings vs Spending
    for agent_type, data in agent_data.items():
        if data:
            earnings = [d['total_earnings'] for d in data]
            spending = [d['total_spending'] for d in data]
            fig.add_trace(
                go.Scatter(
                    x=spending,
                    y=earnings,
                    mode='markers',
                    name=agent_type.replace('_', ' ').title(),
                    marker=dict(color=colors[agent_type], size=8),
                    showlegend=False
                ),
                row=2, col=2
            )
    
    # Update layout
    fig.update_layout(
        title={
            'text': "🔍 Agent Behavior & Economic Pattern Analysis",
            'x': 0.5,
            'font': {'size': 18}
        },
        height=800,
        template='plotly_white'
    )
    
    # Update axis labels
    fig.update_yaxes(title_text="Token Balance", row=1, col=1)
    fig.update_yaxes(title_text="Reputation Score", row=1, col=2)
    fig.update_yaxes(title_text="Satisfaction (%)", row=2, col=1)
    fig.update_yaxes(title_text="Total Earnings", row=2, col=2)
    fig.update_xaxes(title_text="Total Spending", row=2, col=2)
    
    return fig

# Agent analysis button
agent_button = widgets.Button(
    description='🔍 Analyze Agent Behaviors',
    button_style='info',
    layout={'width': '250px', 'height': '40px'}
)

agent_output = widgets.Output()

def on_agent_button_clicked(b):
    with agent_output:
        clear_output(wait=True)
        fig = create_agent_behavior_analysis()
        if fig:
            fig.show()

agent_button.on_click(on_agent_button_clicked)

display(agent_button)
display(agent_output)

## 7. Dashboard Summary

**🎉 PRSM Economic Dashboard Complete!**

This interactive dashboard provides comprehensive Phase 2 economic validation capabilities:

### ✅ **Features Implemented:**
1. **Real-time Economic Simulation** - Mesa framework with 10K+ agents
2. **Interactive Parameter Controls** - Adjust simulation parameters dynamically
3. **Economic Metrics Visualization** - Price, network value, satisfaction, Gini coefficient
4. **Scenario Comparison Analysis** - Compare all market conditions side-by-side
5. **Phase 2 Validation Reporting** - Comprehensive pass/fail validation
6. **Agent Behavior Analytics** - Deep dive into stakeholder patterns

### 📊 **Key Validation Metrics:**
- **Price Stability**: ±20% variance target with dynamic price discovery
- **Incentive Alignment**: 80% stakeholder satisfaction across all types
- **Network Sustainability**: Positive growth rate and economic flow
- **Wealth Distribution**: Gini coefficient ≤ 0.7 for fair distribution

### 🚀 **Phase 2 Status:**
The economic simulation demonstrates functional tokenomics with dynamic price discovery, stakeholder interactions, and network effects. The dashboard enables comprehensive validation of PRSM's economic sustainability across multiple market scenarios.

**Usage Instructions:**
1. Start by running a single simulation with your preferred parameters
2. Generate economic metrics dashboard to visualize results
3. Run scenario comparison across all market conditions
4. Generate Phase 2 validation report for compliance checking
5. Use agent behavior analysis for detailed stakeholder insights

This completes the Interactive Jupyter Dashboard deliverable for Phase 2! 🎯